# SageMaker Processing Jobs With Containers

The objective of this notebook is:
1) Run a processing job to run a scikit-learn script that cleans, pre-processes, performs feature engineering, and splits the input data into train and test sets.

2) Run a training job on the pre-processed training data to train a model

3) Run a processing job on the pre-processed test data to evaluate the trained model's performance

4) Use your own custom container to run processing jobs with your own Python libraries and dependencies.


Source:
https://github.com/aws/amazon-sagemaker-examples/blob/main/sagemaker_processing/scikit_learn_data_processing_and_model_evaluation/scikit_learn_data_processing_and_model_evaluation.ipynb

#### Start Session

In [1]:
from sagemaker.session import Session
from sagemaker import get_execution_role
import sagemaker
import boto3

In [2]:
from sagemaker.sklearn.processing import SKLearnProcessor
import pandas as pd

In [3]:
AWS_ACCESS_KEY = 'Data Structures
AWS_SECRET = 'Algorithms'

region_name='us-east-2'

boto_session = boto3.session.Session(
   aws_access_key_id=AWS_ACCESS_KEY,
   aws_secret_access_key=AWS_SECRET,
   region_name=region_name
)

bucket = 'sm-amazon-nk'

sagemaker_session = Session(boto_session=boto_session)

model_package_group_name = f"AmazonModelPackageGroupName"

role = 'SomeRandomRole'

## Local Run

In [4]:
#instantiate sklearn processor
sklearn_processor = SKLearnProcessor(
    framework_version="0.20.0", role=role, instance_type="ml.m5.xlarge", instance_count=1
)


In [5]:
#input_data = 'https://sm-amazon-nk.s3.us-east-2.amazonaws.com/train_10.csv'
input_data = 'train_10k.csv'

df = pd.read_csv(input_data, index_col=0)
df.head(n=10)

,label,text
0,1,Stuning even for the non-gamerThis sound track...
1,1,The best soundtrack ever to anything.I'm readi...
2,1,Amazing!This soundtrack is my favorite music o...
3,1,Excellent SoundtrackI truly like this soundtra...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."
5,1,an absolute masterpieceI am quite sure any of ...
6,0,"Buyer bewareThis is a self-published book, and..."
7,1,Glorious storyI loved Whisper of the wicked sa...
8,1,A FIVE STAR BOOKI just finished reading Whispe...
9,1,Whispers of the Wicked SaintsThis was a easy t...


In [6]:
#drop duplicates and nan
df.dropna(inplace=True, how='any')
df.drop_duplicates(inplace=True)

##### Check that pre-processing script runs ok before writing it to .py file

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os


X_train, X_test, y_train, y_test = train_test_split(
    df.drop("label", axis=1), 
    df["label"], 
    test_size=0.3, 
    random_state=0
    )

preprocess = TfidfVectorizer(
    max_features=200,
    # strip_accents='ascii',
    # lowercase=True,
    # analyzer = 'word',
    stop_words='english',
    token_pattern = r'(?u)\b\w\w+\b',
    # #max_df=0.95,
    #min_df = 5
                       )
preprocess.fit(X_train['text'])

train_features = preprocess.transform(X_train['text'])
test_features = preprocess.transform(X_test['text'])

print("Train data shape after preprocessing: {}".format(train_features.shape))
print("Test data shape after preprocessing: {}".format(test_features.shape))


Train data shape after preprocessing: (7000, 200)
Test data shape after preprocessing: (3000, 200)


In [8]:
#check shape
train_features.shape

(7000, 200)

In [9]:
#convert to dataframe
train_features_df = pd.DataFrame(train_features.toarray(), columns=preprocess.get_feature_names())
test_features_df = pd.DataFrame(test_features.toarray(), columns=preprocess.get_feature_names())

train_features_df.head()

,10,able,acting,action,actually,ago,album,amazing,amazon,author,...,work,works,world,worst,worth,writing,written,wrong,year,years
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.289452,0.0,0.0,0.0,0.0,0.0,0.0,0.292263,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train_features_df.to_csv('train_features_df.csv', header=True, index=False)
y_train.to_csv('train_labels.csv', header=True, index=False)

In [11]:
train_features_df_local = pd.read_csv('train_features_df.csv')
train_features_df_local.head()

,10,able,acting,action,actually,ago,album,amazing,amazon,author,...,work,works,world,worst,worth,writing,written,wrong,year,years
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.289452,0.0,0.0,0.0,0.0,0.0,0.0,0.292263,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# #get the actual file from an s3 bucket and see how it looks
# train_features_s3_df = pd.read_csv('train_features_from_s3.csv')
# train_features_s3_df.head()

#### Data Preprocessing and Feature Engineering

#### write the actual preprocessing script

In [13]:
%%writefile preprocessing-container.py

import argparse
import os
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.exceptions import DataConversionWarning

warnings.filterwarnings(action="ignore", category=DataConversionWarning)
columns=['label','text']

if __name__ =='__main__':
    #instantiate parser
    parser = argparse.ArgumentParser()
    parser.add_argument('--train-test-split-ratio', 
                        type=float, 
                        default=0.3)
    args, _ = parser.parse_known_args()
    print('Received arguments {}'.format(args))
    
    #read from input file
    input_data_path = os.path.join("/opt/ml/processing/input", "train_10k.csv")
    
    print("Reading input data from {}".format(input_data_path))
    df = pd.read_csv(input_data_path, index_col=0)

    df.dropna(inplace=True, how='any')
    df.drop_duplicates(inplace=True)
    
    #check data ratio
    negative_examples, positive_examples = np.bincount(df["label"])
    print(
        "Data after cleaning: {}, {} positive examples, {} negative examples".format(
            df.shape, positive_examples, negative_examples
        )
    )
    
    #train test split
    split_ratio = args.train_test_split_ratio
    print("Splitting data into train and test sets with ratio {}".format(split_ratio))
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop("label", axis=1), 
        df["label"], 
        test_size=split_ratio, 
        random_state=0
    )
    
    #intantiate preprocess
    preprocess = TfidfVectorizer(
        max_features = 200,
        # strip_accents='ascii', 
        # lowercase=True,
        # analyzer = 'word',
        stop_words='english',
        token_pattern = r'(?u)\b\w\w+\b',
        #max_df=0.95,
        #min_df = 5
    )
    preprocess.fit(X_train['text'])
    
    #run the preprocessing job
    print("Running preprocessing and feature engineering transformations")
    train_features = preprocess.fit_transform(X_train['text'])
    test_features = preprocess.transform(X_test['text'])
    
    #convert to dataframe
    train_features_df = pd.DataFrame(train_features.toarray(), columns=preprocess.get_feature_names())
    test_features_df = pd.DataFrame(test_features.toarray(), columns=preprocess.get_feature_names())
    
    print("Train data shape after preprocessing: {}".format(train_features_df.shape))
    print("Test data shape after preprocessing: {}".format(test_features_df.shape))

    train_features_output_path = os.path.join("/opt/ml/processing/train", "train_features.csv")
    train_labels_output_path = os.path.join("/opt/ml/processing/train", "train_labels.csv")

    test_features_output_path = os.path.join("/opt/ml/processing/test", "test_features.csv")
    test_labels_output_path = os.path.join("/opt/ml/processing/test", "test_labels.csv")
    
    #drop header from dfs
    print("Saving training features to {}".format(train_features_output_path))
    train_features_df.to_csv(train_features_output_path, header=True, index=False)

    print("Saving test features to {}".format(test_features_output_path))
    test_features_df.to_csv(test_features_output_path, header=True, index=False)

    print("Saving training labels to {}".format(train_labels_output_path))
    y_train.to_csv(train_labels_output_path, header=True, index=False)

    print("Saving test labels to {}".format(test_labels_output_path))
    y_test.to_csv(test_labels_output_path, header=True, index=False)
    
    

Overwriting preprocessing-container.py


In [14]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(
    code="preprocessing-container.py",
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    if output["OutputName"] == "train_data":
        preprocessed_training_data = output["S3Output"]["S3Uri"]
    if output["OutputName"] == "test_data":
        preprocessed_test_data = output["S3Output"]["S3Uri"]


Job Name:  sagemaker-scikit-learn-2022-07-21-16-06-29-125
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-06-29-125/input/input-1/train_10k.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-06-29-125/input/code/preprocessing-container.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-06-29-125/output/train_data', 'LocalPath': '/opt/ml/processing

In [15]:
preprocessed_training_data

's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-06-29-125/output/train_data'

In [16]:
#check where data is saved
preprocessed_training_data + "/train_features.csv"

's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-06-29-125/output/train_data/train_features.csv'

In [17]:
s3 = boto_session.client('s3')
obj = s3.get_object(Bucket='sagemaker-us-east-2-013747046745',
                    Key='sagemaker-scikit-learn-2022-06-23-19-20-36-300/output/train_data/train_features.csv')
training_features = pd.read_csv(obj['Body'])
training_features.head()

,10,able,acting,action,actually,ago,album,amazing,amazon,author,...,work,works,world,worst,worth,writing,written,wrong,year,years
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.391334,0.000000,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.360004,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.149208,0.0,0.0,0.000000,0.000000,0.0,0.169587,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.161226,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0


In [18]:
# # #inspect the output of the pre-processing job, which consists of the processed features.
# training_features = pd.read_csv(preprocessed_training_data + "/train_features.csv", nrows=10)
# print("Training features shape: {}".format(training_features.shape))
# training_features.head(n=10)

### Training using the pre-processed data


In [19]:
from sagemaker.sklearn.estimator import SKLearn

sklearn = SKLearn(
    entry_point="train.py", framework_version="0.20.0", instance_type="ml.m5.xlarge", role=role
)

In [20]:
%%writefile train.py

import os

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

if __name__ == "__main__":
    training_data_directory = "/opt/ml/input/data/train"
    train_features_data = os.path.join(training_data_directory, "train_features.csv")
    train_labels_data = os.path.join(training_data_directory, "train_labels.csv")
    print("Reading input data")
    X_train = pd.read_csv(train_features_data)
    y_train = pd.read_csv(train_labels_data)

    model = LogisticRegression(class_weight="balanced", solver="lbfgs")
    print("Training LR model")
    model.fit(X_train, y_train)
    model_output_directory = os.path.join("/opt/ml/model", "model.joblib")
    print("Saving model to {}".format(model_output_directory))
    joblib.dump(model, model_output_directory)

Overwriting train.py


In [21]:
sklearn.fit({"train": preprocessed_training_data})
training_job_description = sklearn.jobs[-1].describe()
model_data_s3_uri = "{}{}/{}".format(
    training_job_description["OutputDataConfig"]["S3OutputPath"],
    training_job_description["TrainingJobName"],
    "output/model.tar.gz",
)

2022-07-21 16:11:10 Starting - Starting the training job...
2022-07-21 16:11:33 Starting - Preparing the instances for trainingProfilerReport-1658419869: InProgress
......
2022-07-21 16:12:34 Downloading - Downloading input data...
2022-07-21 16:13:14 Training - Downloading the training image...
2022-07-21 16:13:48 Uploading - Uploading generated training model
2022-07-21 16:13:48 Completed - Training job completed
2022-07-21 16:13:34,015 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-07-21 16:13:34,018 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-21 16:13:34,030 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-07-21 16:13:34,423 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-21 16:13:34,434 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-07-21 16:13:34,445 sagemaker-training-toolki

### Model Evaluation


In [22]:
%%writefile evaluation.py

import json
import os
import tarfile

import pandas as pd

from sklearn.externals import joblib
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

if __name__ == "__main__":
    model_path = os.path.join("/opt/ml/processing/model", "model.tar.gz")
    print("Extracting model from path: {}".format(model_path))
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    print("Loading model")
    model = joblib.load("model.joblib")

    print("Loading test input data")
    test_features_data = os.path.join("/opt/ml/processing/test", "test_features.csv")
    test_labels_data = os.path.join("/opt/ml/processing/test", "test_labels.csv")

    X_test = pd.read_csv(test_features_data, header='infer')
    y_test = pd.read_csv(test_labels_data, header='infer')
    predictions = model.predict(X_test)

    print("Creating classification evaluation report")
    report_dict = classification_report(y_test, predictions, output_dict=True)
    report_dict["accuracy"] = accuracy_score(y_test, predictions)
    report_dict["roc_auc"] = roc_auc_score(y_test, predictions)

    print("Classification report:\n{}".format(report_dict))

    evaluation_output_path = os.path.join("/opt/ml/processing/evaluation", "evaluation.json")
    print("Saving classification report to {}".format(evaluation_output_path))

    with open(evaluation_output_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting evaluation.py


In [23]:
import json
from sagemaker.s3 import S3Downloader

sklearn_processor.run(
    code="evaluation.py",
    inputs=[
        ProcessingInput(source=model_data_s3_uri, destination="/opt/ml/processing/model"),
        ProcessingInput(source=preprocessed_test_data, destination="/opt/ml/processing/test"),
    ],
    outputs=[ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation")],
)
evaluation_job_description = sklearn_processor.jobs[-1].describe()


Job Name:  sagemaker-scikit-learn-2022-07-21-16-14-02-135
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-11-08-217/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-06-29-125/output/test_data', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-scikit-learn-2022-07-21-16-14-02-135/input/code/evaluation.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3I

##### Now retrieve the file evaluation.json from Amazon S3, which contains the evaluation report.


In [24]:
evaluation_output_config = evaluation_job_description["ProcessingOutputConfig"]
for output in evaluation_output_config["Outputs"]:
    if output["OutputName"] == "evaluation":
        evaluation_s3_uri = output["S3Output"]["S3Uri"] + "/evaluation.json"
        break

evaluation_output = S3Downloader.read_file(evaluation_s3_uri)
evaluation_output_dict = json.loads(evaluation_output)
print(json.dumps(evaluation_output_dict, sort_keys=True, indent=4))

{
    "0": {
        "f1-score": 0.7936350074589756,
        "precision": 0.7869822485207101,
        "recall": 0.8004012036108324,
        "support": 997
    },
    "1": {
        "f1-score": 0.7913524384112619,
        "precision": 0.7981744421906694,
        "recall": 0.7846460618145563,
        "support": 1003
    },
    "accuracy": 0.7925,
    "macro avg": {
        "f1-score": 0.7924937229351188,
        "precision": 0.7925783453556897,
        "recall": 0.7925236327126943,
        "support": 2000
    },
    "micro avg": {
        "f1-score": 0.7925,
        "precision": 0.7925,
        "recall": 0.7925,
        "support": 2000
    },
    "roc_auc": 0.7925236327126944,
    "weighted avg": {
        "f1-score": 0.7924902990815471,
        "precision": 0.7925951336461946,
        "recall": 0.7925,
        "support": 2000
    }
}


### Running processing jobs with FrameworkProcessor to include custom dependencies


In [25]:
from sagemaker.processing import FrameworkProcessor

est_cls = sagemaker.sklearn.estimator.SKLearn
framework_version_str="0.20.0"

script_processor = FrameworkProcessor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    estimator_cls=est_cls,
    framework_version=framework_version_str
)

In [43]:
# copy the previously created preprocessing.py script to code folder for next run method
!mkdir code
!cp preprocessing-container.py ./code
!cp train.py ./code


mkdir: code: File exists


In [27]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

script_processor.run(
    code="preprocessing-container.py",
    source_dir="code",
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)


Job Name:  sklearn-2022-07-21-16-18-31-353
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sklearn-2022-07-21-16-18-31-353/input/input-1/train_10k.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sklearn-2022-07-21-16-18-31-353/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sklearn-2022-07-21-16-18-31-353/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributi

## build custom container from scratch

In [28]:
#By default optional steps do not run automatically, set run_optional_steps to True if you want to execute optional steps
run_optional_steps = False


In [29]:
# This will stop the below cells from executing if "Run All Cells" was used on the notebook.
if not run_optional_steps:
    raise SystemExit("Stop here. Do not automatically execute optional steps.")

SystemExit: Stop here. Do not automatically execute optional steps.

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [30]:
!mkdir docker

mkdir: docker: File exists


In [31]:
%%writefile docker/Dockerfile

#!/usr/bin/env bash

FROM python:3.8-slim-buster

RUN pip3 install pandas==1.1.5 scikit-learn==0.24.2
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

Overwriting docker/Dockerfile


#### Buld the container using the docker command, creates an Amazon Elastic Container Registry (Amazon ECR) repository, and pushes the image to Amazon ECR.



In [32]:
#set region
region = region_name
region

'us-east-2'

In [33]:
account_id = boto_session.client('sts').get_caller_identity().get("Account")
ecr_repository = "sagemaker-processing-container"
tag = ":latest"

uri_suffix = "amazonaws.com"
if region in ["cn-north-1", "cn-northwest-1"]:
    uri_suffix = "amazonaws.com.cn"
processing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
    account_id, region, uri_suffix, ecr_repository + tag
)

In [34]:
#check account number
account_id = boto_session.client('sts').get_caller_identity().get("Account")
print(account_id)

013747046745


In [35]:
processing_repository_uri

'013747046745.dkr.ecr.us-east-2.amazonaws.com/sagemaker-processing-container:latest'

In [36]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository docker --platform=linux/amd64 

[+] Building 0.0s (0/1)                                                         
[+] Building 0.1s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 197B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  0.0s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 197B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

[+] Building 2.2s (3/4)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 197B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  2.1s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
[+] Building 2.4s (4/4)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 197B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docke

[+] Building 3.3s (4/6)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 197B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  2.2s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [1/2] FROM docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.9s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.0s
 => => sha256:92043b8eb059a431b5c6a6f831d39dca1fccb41e6bc 7.53kB / 7.53kB  0.0s
 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 14.68MB / 27.14MB  0.9s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcb

 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 27.14MB / 27.14MB  1.3s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcbea1ab570483c89a8 11.29MB / 11.29MB  0.6s
 => => sha256:5f22b4140ef49429890f3ea95abf87f6baf96cfb44b 1.86kB / 1.86kB  0.0s
 => => sha256:b878b6e28df2998d4a473137d1a1809c1127d9aba14 1.37kB / 1.37kB  0.0s
 => => sha256:d49a7ac5e6c0f57e9760d2bd15ddd99b8710e8bf8702f07 232B / 232B  0.8s
 => => sha256:ac10782b084356a233b098aa4a29fdef03e992172d2 3.17MB / 3.17MB  1.0s
 => => extracting sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b58456b691ba  0.2s
[+] Building 4.1s (4/6)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 197B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B        

[+] Building 4.7s (4/6)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 197B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  2.2s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [1/2] FROM docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  2.3s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.0s
 => => sha256:92043b8eb059a431b5c6a6f831d39dca1fccb41e6bc 7.53kB / 7.53kB  0.0s
 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 27.14MB / 27.14MB  1.3s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcb

...

[+] Building 81.1s (5/6)                                                        
 => [1/2] FROM docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  3.0s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.0s
 => => sha256:92043b8eb059a431b5c6a6f831d39dca1fccb41e6bc 7.53kB / 7.53kB  0.0s
 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 27.14MB / 27.14MB  1.3s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcbea1ab570483c89a8 11.29MB / 11.29MB  0.6s
 => => sha256:5f22b4140ef49429890f3ea95abf87f6baf96cfb44b 1.86kB / 1.86kB  0.0s
 => => sha256:b878b6e28df2998d4a473137d1a1809c1127d9aba14 1.37kB / 1.37kB  0.0s
 => => sha256:d49a7ac5e6c0f57e9760d2bd15ddd99b8710e8bf8702f07 232B / 232B  0.8s
 => => sha256:ac10782b084356a233b098aa4a29fdef03e992172d2 3.17MB / 3.17MB  1.0s
 => => extracting sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b58456b691ba  0.8s
 => => extracting sha256:64d0f1d7987a21

[+] Building 81.7s (5/6)                                                        
 => [1/2] FROM docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  3.0s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.0s
 => => sha256:92043b8eb059a431b5c6a6f831d39dca1fccb41e6bc 7.53kB / 7.53kB  0.0s
 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 27.14MB / 27.14MB  1.3s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcbea1ab570483c89a8 11.29MB / 11.29MB  0.6s
 => => sha256:5f22b4140ef49429890f3ea95abf87f6baf96cfb44b 1.86kB / 1.86kB  0.0s
 => => sha256:b878b6e28df2998d4a473137d1a1809c1127d9aba14 1.37kB / 1.37kB  0.0s
 => => sha256:d49a7ac5e6c0f57e9760d2bd15ddd99b8710e8bf8702f07 232B / 232B  0.8s
 => => sha256:ac10782b084356a233b098aa4a29fdef03e992172d2 3.17MB / 3.17MB  1.0s
 => => extracting sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b58456b691ba  0.8s
 => => extracting sha256:64d0f1d7987a21

[+] Building 82.2s (5/6)                                                        
 => [1/2] FROM docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  3.0s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.0s
 => => sha256:92043b8eb059a431b5c6a6f831d39dca1fccb41e6bc 7.53kB / 7.53kB  0.0s
 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 27.14MB / 27.14MB  1.3s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcbea1ab570483c89a8 11.29MB / 11.29MB  0.6s
 => => sha256:5f22b4140ef49429890f3ea95abf87f6baf96cfb44b 1.86kB / 1.86kB  0.0s
 => => sha256:b878b6e28df2998d4a473137d1a1809c1127d9aba14 1.37kB / 1.37kB  0.0s
 => => sha256:d49a7ac5e6c0f57e9760d2bd15ddd99b8710e8bf8702f07 232B / 232B  0.8s
 => => sha256:ac10782b084356a233b098aa4a29fdef03e992172d2 3.17MB / 3.17MB  1.0s
 => => extracting sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b58456b691ba  0.8s
 => => extracting sha256:64d0f1d7987a21

[+] Building 82.8s (6/7)                                                        
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  2.2s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [1/2] FROM docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  3.0s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.0s
 => => sha256:92043b8eb059a431b5c6a6f831d39dca1fccb41e6bc 7.53kB / 7.53kB  0.0s
 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 27.14MB / 27.14MB  1.3s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcbea1ab570483c89a8 11.29MB / 11.29MB  0.6s
 => => sha256:5f22b4140ef49429890f3ea95abf87f6baf96cfb44b 1.86kB / 1.86kB  0.0s
 => => sha256:b878b6e28df2998d4a473137d

[+] Building 83.4s (6/7)                                                        
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  2.2s
 => [auth] library/python:pull token for registry-1.docker.io              0.0s
 => [1/2] FROM docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  3.0s
 => => resolve docker.io/library/python:3.8-slim-buster@sha256:5f22b4140e  0.0s
 => => sha256:92043b8eb059a431b5c6a6f831d39dca1fccb41e6bc 7.53kB / 7.53kB  0.0s
 => => sha256:ac2fb615420c18b61e0693f2569a3d38e3b9b5845 27.14MB / 27.14MB  1.3s
 => => sha256:64d0f1d7987a219c326033c9cf223e79135368247fb 2.78MB / 2.78MB  0.7s
 => => sha256:b650c79550ecb753015f67dcbea1ab570483c89a8 11.29MB / 11.29MB  0.6s
 => => sha256:5f22b4140ef49429890f3ea95abf87f6baf96cfb44b 1.86kB / 1.86kB  0.0s
 => => sha256:b878b6e28df2998d4a473137d

In [37]:
#Authenticate to your default registry
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 013747046745.dkr.ecr.us-east-2.amazonaws.com/sagemaker-processing-container.dkr.ecr.us-east-2.amazonaws.com

Login Succeeded


In [38]:
#create repo
#error message OK if already exists
!aws ecr create-repository --repository-name $ecr_repository


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-processing-container' already exists in the registry with id '013747046745'


In [39]:
!docker tag {ecr_repository + tag} $processing_repository_uri #repo tag

In [40]:
!docker push $processing_repository_uri

The push refers to repository [013747046745.dkr.ecr.us-east-2.amazonaws.com/sagemaker-processing-container]

079d9333: Preparing 
6fd10755: Preparing 
f39a5c2e: Preparing 
9397ed05: Preparing 
b6c0b614: Preparing 
079d9333: Pushed   465.5MB/458.6MBPushing  313.9MB/458.6MBPushing  453.7MB/458.6MBlatest: digest: sha256:cdb5505e6bbd8955cb6fe3ed5a3757be2b90032c24e257eda42c132c6074981e size: 1583


In [88]:
from sagemaker.processing import ScriptProcessor

script_processor = ScriptProcessor(
    command=["python3"],
    image_uri=processing_repository_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)

In [89]:
script_processor.run(
    code="preprocessing-container.py",
    inputs=[ProcessingInput(source=input_data, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="test_data", source="/opt/ml/processing/test"),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)


Job Name:  sagemaker-processing-container-2022-07-22-22-14-52-636
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-processing-container-2022-07-22-22-14-52-636/input/input-1/train_10k.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-processing-container-2022-07-22-22-14-52-636/input/code/preprocessing-container.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-processing-container-2022-07-22-22-14-52-636/output/train_data', 

In [91]:
script_processor.jobs[-1].describe()

{'ProcessingInputs': [{'InputName': 'input-1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-processing-container-2022-07-22-22-14-52-636/input/input-1/train_10k.csv',
    'LocalPath': '/opt/ml/processing/input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'code',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-processing-container-2022-07-22-22-14-52-636/input/code/preprocessing-container.py',
    'LocalPath': '/opt/ml/processing/input/code',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'train_data',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-2-013747046745/sagemaker-processing-container-2022-07-22-22-14-52-636/output/t

### Running Training Job

In [62]:
ecr_repository_train = "sagemaker-train-container"
tag = ":latest"

uri_suffix = "amazonaws.com"
if region in ["cn-north-1", "cn-northwest-1"]:
    uri_suffix = "amazonaws.com.cn"
    
train_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
    account_id, region, uri_suffix, ecr_repository_train + tag
)

In [63]:
train_repository_uri

'013747046745.dkr.ecr.us-east-2.amazonaws.com/sagemaker-train-container:latest'

In [64]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository_train docker --platform=linux/amd64 

[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/library/python:3.8-slim-buster  0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 37B                                        0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

In [65]:
#Authenticate to your default registry
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 013747046745.dkr.ecr.us-east-2.amazonaws.com/sagemaker-train-container.dkr.ecr.us-east-2.amazonaws.com

Login Succeeded


In [66]:
#create repo
#error message OK if already exists
!aws ecr create-repository --repository-name $ecr_repository_train


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'sagemaker-train-container' already exists in the registry with id '013747046745'


In [67]:
!docker tag {ecr_repository_train + tag} $train_repository_uri #repo tag

In [68]:
!docker push $train_repository_uri

The push refers to repository [013747046745.dkr.ecr.us-east-2.amazonaws.com/sagemaker-train-container]

079d9333: Preparing 
6fd10755: Preparing 
f39a5c2e: Preparing 
9397ed05: Preparing 
b6c0b614: Preparing 
079d9333: Pushed   465.5MB/458.6MB2KPushing    441MB/458.6MBlatest: digest: sha256:cdb5505e6bbd8955cb6fe3ed5a3757be2b90032c24e257eda42c132c6074981e size: 1583


In [69]:
train_repository_uri

'013747046745.dkr.ecr.us-east-2.amazonaws.com/sagemaker-train-container:latest'

In [70]:
#run the training job
script_processor_train = ScriptProcessor(
    command=["python3"],
    image_uri=train_repository_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)

In [1]:
# script_processor_train.run(
#     code="train.py",
#     inputs=[ProcessingInput(source=preprocessed_training_data, destination="/opt/ml/processing/input")],
#     outputs=[
#         ProcessingOutput(output_name="train_data_pred", source="/opt/ml/processing/train"),
#     ],
# )


In [ ]:
# script_processor_train_description = script_processor_train.jobs[-1].describe()
# print(script_processor_train_description)